<a href="https://colab.research.google.com/github/Rai90/ggepy-tutorial/blob/main/gge_py_2_mapping_function_across_collection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Authentication**

In [2]:
import ee

# Trigger the authentication flow.
ee.Authenticate()



In [3]:
import geemap
ee.Initialize(project='ee-raischwalbert')

# Get Landsat 8 imagery over Manhattan KS from the last 2 years

In [95]:

# Define the region of interest as Manhattan, Kansas.
region = ee.Geometry.Rectangle([-96.65, 39.1, -96.5, 39.2]).centroid()

# Filter the Landsat 8 imagery collection for the last two years.
landsat8 = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2") \
  .filterBounds(region) \
  .filterDate('2022-01-01', '2024-01-01')

# Get a single image from the collection and rescale to reflectance values
image = ee.Image(landsat8.toList(10).get(9)).select('SR_B.').multiply(0.0000275).add(-0.2)


visualization = {
    'bands': ['SR_B4', 'SR_B3', 'SR_B2'],
    'min': 0.0,
    'max': 0.3,
}



# Display the image.
Map = geemap.Map()
Map.addLayer(image, visualization, 'True Color (432)')
Map.centerObject(region, 10)
Map


Map(center=[39.1500121927059, -96.5749999999996], controls=(WidgetControl(options=['position', 'transparent_bg…

In [28]:
#check all images within the collection

display(landsat8)

# Understanding bitwise operations



*   **000000001000** means *cloudShadowBitMask*
*   **000000100000** means *cloudsBitMask*





# Map

Use map to apply function across collection

In [96]:
# prompt: remove cloudy pixels from landsat8 collection

# Mask cloudy pixels in the Landsat 8 imagery.
#landsat8_masked = landsat8.map(lambda image: image.mask(image.select('QA_PIXEL').bitwiseAnd(1 << 3).eq(0)))
def cloud_removal(x):
  aa = x.select('QA_PIXEL').bitwiseAnd(1 << 3).eq(0).And(x.select('QA_PIXEL').bitwiseAnd(1 << 5).eq(0))
  return x.mask(aa)

landsat8_masked = landsat8.map(clould_removal)


# Get a single image from the masked collection.
image_masked = ee.Image(landsat8_masked.toList(10).get(9)).select('SR_B.').multiply(0.0000275).add(-0.2)

# Display the image.

Map.addLayer(image_masked,visualization,"Cloud Masked")
Map.centerObject(region, 10)
Map



Map(bottom=1746.0, center=[39.1500121927059, -96.5749999999996], controls=(WidgetControl(options=['position', …

In [98]:
# prompt: calculate NDVI over landsat8_masked collection

def addNDVI(image):
  ndvi = image.normalizedDifference(['SR_B5', 'SR_B4']).rename('NDVI')
  return image.addBands(ndvi)

landsat8_masked_ndvi = landsat8_masked.map(addNDVI)

display(landsat8_masked_ndvi)



In [99]:
# Display the image.
image_ndvi = ee.Image(landsat8_masked_ndvi.toList(10).get(9)).select(["NDVI"])

viz_params_ndvi = {
    'min': 0,
    'max': 0.9,
    'palette': [
        'FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718',
        '74A901', '66A000', '529400', '3E8601', '207401', '056201',
        '004C00', '023B01', '012E01', '011D01', '011301'
    ],
}


Map.addLayer(image_ndvi,viz_params_ndvi)
Map.centerObject(region, 10)
Map


Map(bottom=199484.0, center=[39.1500121927059, -96.5749999999996], controls=(WidgetControl(options=['position'…

# **Iterate**

With map we can apply a function over one collection. Iterate is the solution we have when the function requires multiple collections as inputs